computing transitive closure on a relation

Review slides https://pages.mtu.edu/~nilufer/classes/cs2311/2012-march/cs2311-s12-ch9-relations-part2.pdf#slide.1
This clarifies that transitive closure is something we can compute by thinking of a relation as a graph.
It also clarifies that the complete transitive closure is the power of the matrix raised to n (number of elements) because that covers all paths up to length n.

A relation can be visualized as a bipartite (two part) graph. 
In [[https://rpubs.com/pjmurphy/317838][R we can visualize a bipartite graph]] directly with plot function.
This builds on [[https://www.rdocumentation.org/packages/igraph/versions/1.2.4.2/topics/graph_from_adjacency_matrix][creating a graph from the adjacency matrix.]]
When we have two distinct sets, we need to construct the adjacency matrix that has vertices combined from all the elements of both sets.
It's easy to [[https://en.wikipedia.org/wiki/Adjacency_matrix#Of_a_bipartite_graph][construct this adjacency matrix of a bipartite graph]].

Viewed as a graph, relations are networks.
This is a powerful visualization of relations and allows us to visually explore many familiar networks like the web or social networks.
We can draw these networks in R.
[[https://hal.archives-ouvertes.fr/hal-01722543/][It might be easier to use the ggplot2 routines]].  https://hal.archives-ouvertes.fr/hal-01722543/
Or, once defined in a standard format, a dedicated tool for network visualization.
[[https://medium.com/@Elise_Deux/list-of-free-graph-visualization-applications-9c4ff5c1b3cd][There are many to choose from.]]


We learn by discovering relationships between ideas.
We can document these connections using the notation of sets and recording the relations between items in those sets.
Combining these relations can help us discover new connections between items.

As a simple example, if we have a set of people { a, b, c, d } and know a parent-child relation, R = {(a,b)},  and sibling relations, S={(b,c),(b,d)},  we can compose the relation of aunts or uncles, RS={(a,c),(a,d)}.
An aunt (or uncle) is a person who is a sibling of a parent.
Therefore, if we know a is a "child of" b, aRb, and b is "sibling of" c and d,  bSc and bSd, then the composition of the relation tells us which people have aunt or uncle relationships.

This composition describes a transitive relationship, where if a is related to b and b related to c then a is related to c.
Transitive relationships can be composed of any number of steps.
For example, we could have a transitive relationship between a and d of length three where a is related to b, b is related to c, and c is related to d.

Computing the transitive closure of a set means identifying all the transitive relationships in a set.
A closure is just the complete collection of all the relationships in which we are interested, the transitive ones in this case.

Relations can be represented as a matrix.
The n items in the set define the n-rows and n-columns of an nxn matrix.
The relation is represented by putting a one in the matrix entry i,j if there is a relation between element i and j of the set and a zero if there is not.
This gives us a binary matrix of 1's and 0's.

We can use matrix multiplication to compute transitive relations.
If we have two relations R and S over a set of n items, then we an represent each relation in its nxn binary matrix form as M_R and M_S.
The computed transitive relation RS is then the multiplication of M_R and M_S,  M_R * M_S = M_RS.
This particular computation gives the transitive relations of length two, i.e. aRb and bSc.

More generally, we can compute transitive relations of any length by repeated matrix multiplication.
If we have a relation R, we can compute the transitive closure R*, the complete collection of relationships of any length between items in the set.
Each multiplication increases the path length by one.
This makes sense.
If we start with the matrix M_R, it contains all the relations of a length one.
That is, direct relationships between the items in the set.
If we multiply M_R * M_R, we get the transitive relations of length two.
If we do it again and multiply (M_R * M_R) * M_R we get the transitive relations of length three.
The longest path of interest is n steps away, a transitive relation composed of steps through each item in the set.
This means we can compute the transitive closure of a relation R on an n-item set by multiplying the M_R matrix by itself n times.
This is simply the nth power of the matrix.
Therefore the transitive closure R* can be computed as M_(R*) = (M_R)^n.

Note, since we are really just interested in connectivity, we need to do a little data clean up at the end.
The matrix multiplication will actually result in values greater than 1 for nodes that are connected by multiple nodes.
If we just interested in connectivity we can just normalize all non-zero elements to one.
Alternatively we can define binary matrix multiplication using Boolean ANDs for multiplication and OR for addtion.
This  makes sure the values remain 1 and 0.

There's another perspective on the relation-as-a-matrix representation that makes the transitive closure extremely useful to a broad set of applications.
We can represent a graph of n verticies as an nxn adjacency matrix.
The adjacency matrix is a matrix with values of 1 for vertices i,j that have an edge connecting them in the graph and values of 0 when there is no edge.
A binary matrix that maps elements of a set according to some relation is identical to an adjacency matrix for a graph.
This means we can represent interesting graphs like social networks or maps of the web as adjacency matrix.
We can then compute the transitive closure of these relations and find all nodes that are connected to each other either directly or through intermediate nodes.
Limiting the path length, for example to two, would make easy work of determining who is connected by a friend-of-friends relation.
This is a powerful tool for communitity detection in graphs.

So, is it practical to just multiply the matrices together to compute the transitive closure?
Whenever we do work on a computer we should be interested in, meaning concerned about, the amount of time it takes to do that work.
If our data sizes are small, then we might get away with a naive approach for computing transitive closure.
By naive we mean a brute force computation that does all the multiplication and addition steps you'd expect to do for matrix multiplication.
Multiplying two nxn matrices means n^3 operations, so the naive approach is bounded by O(n^3).
But since we have to do n of these matrix multiplications, that really works out to O(n^4).

Deciding of our data is too big depends on that computation decision.
If we have 100 elements in our set, then a naive computation of transitive closure would be 100^4 or 100million operations.
Probably not impossible to wait for, but it increases exponentially for every factor of 10 increase in our set of elements.
Data becomes big pretty quickly at that rate.
A one-thousand node network would take 1000 times longer than the 100million operations that it took for our 100 element set.
This could quickly start to feel like waiting or, worse, limit the amount of information we are able to consider.

With social networks or the web graph we easily imagine thousands, millions or even billions of nodes in our network.
Even if we are only interested in nodes connected by one or two intermediate nodes we are still bounded by O(n^3).
The naive approach quickly becomes unmanagable at the scale of thousands of nodes.

In 1969 Strassen introduced the idea of fast matrix multiplication and was able to demonstrate performance bounded by O(n^(2.8)).
These methods have been improved and the state of the art for fast matrix multiply is now about O(n^(2.373)).
Keeping in mind that if we want the transitive closure we need still need to do this n times, bumping us above O(n^3).

Are there faster methods?
The well-known Floyd-Warshall algorithm for computing the shortest path between all pairs of nodes in a weighted graph can be applied to this problem.
It runs in O(n^3), for n-node networks.
We don't really care about weights for transitive closure, but finding the cheapest path is a nice feature to have in real world networks.
Floyd-Warshall does this using a dynamic program technique to keep the computational bound at O(n^3).
It inspects paths of increasing length between each node and keeps track of the shortest one.
Inspecting each of the n^2 node pairs in the graph n times keeps it bound at at O(n^3).
Because it finds the shortest path between all pairs, we end up with a complete set of paths across the graph at the end of the compution.
That is we have the transitive closure in O(n^3) operations.

Let's revisit the earlier observation that we are working with binary matrices we can therefore use binary matrix multiplication.
We replace the element-wise multiplication by Boolean AND operations and sum by OR operations.
This means the value for a cell in the result matrix will be 1 if there is a 1 in any of the matching i,j and j,i cells.
The Boolean OR operations means the result will be 1 if any one of these cell-level operations for the row and column multiplication is one.

We can rewrite our left-side matrix of the multiplication as set of rows that just records the postion of the ones in that row.
We then step through each element of these these i rows and inspect the corresponding element for its position k in the right side matrix in cell k,j.
If the we find a 1 in that position then we know there is a path and set the value of the resulting cell to 1.
We can then move on to the next row.
The expected time for most matrices is O(n^2).
There are worst case peformance of O(n^3) but that's not any worse than the solutions above.

This gives use an O(n^2) algorithm for computing paths between nodes.
This means we can compute a 1-million node network in same time it took us for computing a 1000 node network with our naive approach above.
That's a huge increase in the size of networks we can explore.
There's a log of value in thinking about perfomance when working with computers.




